In [1]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import Language
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

# Python 파일을 로드하고 문서를 분할합니다.
repo_root = "/Users/cosmos/Desktop/ai/langchain/libs"
repo_core = repo_root + "/core/langchain_core"
repo_community = repo_root + "/community/langchain_community"
repo_experimental = repo_root + "/experimental/langchain_experimental"
repo_partners = repo_root + "/partners"
repo_text_splitter = repo_root + "/text_splitters/langchain_text_splitters"
repo_cookbook = repo_root + "/cookbook"

py_documents = []
for path in [repo_core, repo_community, repo_experimental, repo_partners, repo_cookbook]:
    loader = GenericLoader.from_filesystem(
        path, glob="**/*", suffixes=[".py"],
        parser=LanguageParser(language=Language.PYTHON, parser_threshold=30),
    )
    py_documents.extend(loader.load())
print(f".py 파일의 개수: {len(py_documents)}")

py_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
py_docs = py_splitter.split_documents(py_documents)
print(f"분할된 .py 파일의 개수: {len(py_docs)}")

# MDX 파일을 로드하고 문서를 분할합니다.
root_dir = "/Users/cosmos/Desktop/ai/langchain/"

mdx_documents = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if (file.endswith(".mdx")) and "*venv/" not in dirpath:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                mdx_documents.extend(loader.load())
            except Exception:
                pass
print(f".mdx 파일의 개수: {len(mdx_documents)}")

mdx_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
mdx_docs = mdx_splitter.split_documents(mdx_documents)
print(f"분할된 .mdx 파일의 개수: {len(mdx_docs)}")

# Teddy님의 랭체인노트를 로드하고 문서를 분할합니다.
import pandas as pd
from langchain.schema import Document

df = pd.read_csv('data_list_with_content.csv')
df_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
teddy_docs = []
for index, row in df.iterrows():
    if pd.isna(row['content']):
        continue
    chunks = df_splitter.split_text(row['content'])
    for chunk in chunks:
        teddy_docs.append(Document(page_content=chunk, metadata={"title": row['title'], "source": row['source']}))
print(f"분할된 .df 파일 개수: {len(teddy_docs)}")

# 파이썬 문서, MDX 문서, PDF 문서, 테디노트(Langhchin-KR) 문서를 결합합니다.
combined_documents = py_docs + mdx_docs + teddy_docs
print(f"총 도큐먼트 개수((teddy, py, mdx)): {len(combined_documents)}")

.py 파일의 개수: 5818
분할된 .py 파일의 개수: 10177
.mdx 파일의 개수: 272
분할된 .mdx 파일의 개수: 597
분할된 .df 파일 개수: 825
총 도큐먼트 개수((teddy, py, mdx)): 11599


In [2]:

# 필요한 임베딩과 캐싱설정을 수행합니다. 
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", disallowed_special=())
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace=embeddings.model)

In [3]:
# Kiwi Tokenizer를 설정합니다.
from kiwipiepy import Kiwi

kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# FAISS 클래스를 가져와 검색 모델 인스턴스를 생성합니다.
from langchain_community.vectorstores import FAISS, Chroma

FAISS_DB_INDEX = "langchain_faiss"
db = FAISS.from_documents(combined_documents, cached_embeddings)
faiss_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})

# BM25Retriever 클래스를 가져와 검색 모델 인스턴스를 생성합니다.
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(combined_documents, preprocess_func=kiwi_tokenize, k=10)

# EnsembleRetriever 클래스를 사용하여 검색 모델을 결합하여 사용합니다.
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.6, 0.4], search_type="mmr",
)

In [4]:
# Prompt를 설정합니다.
from langchain_core.prompts import (
    PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate
)

prompt_template = """
당신은 20년차 AI 개발자입니다. 당신의 임무는 주어진 질문에 대하여 최대한 문서의 정보를 활용하여 답변하는 것입니다.
문서는 Python 코드에 대한 정보를 담고 있습니다. 따라서, 답변을 작성할 때에는 Python 코드에 대한 상세한 code snippet을 포함하여 작성해주세요.
최대한 자세하게 답변하고, 한글로 답변해 주세요. 주어진 문서에서 답변을 찾을 수 없는 경우, "문서에 답변이 없습니다."라고 답변해 주세요.
답변에 대한 출처(source)를 반드시 표기해야 합니다. 출처(source)는 당신이 답변을 만들때 참조한 문서의 metadata의 source를 표기해 주세요.

# 주어진 문서:
{context}

# 질문:
{question}

# 답변: 

# 출처:
- source1
- source2
- ...
"""
def get_context(query):
    relevant_docs = ensemble_retriever.invoke(query)
    return relevant_docs

system_template = SystemMessagePromptTemplate.from_template(prompt_template)
system_message = system_template.format(context=get_context("{question}"), question="{question}")

prompt = ChatPromptTemplate.from_messages([
    system_message,
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{question}")
])

In [5]:

# Callback, LLM, Chain를 설정합니다.
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

class StreamCallback(BaseCallbackHandler):
     def on_llm_new_token(self, token: str, **kwargs): print(token, end="", flush=True)

llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True, verbose=True, callbacks=[StreamCallback()])

# 대화기록 유지를 위한 함수입니다.
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

runnable = prompt | llm | StrOutputParser()

store = {}  # 세션 기록을 저장할 딕셔너리

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

rag_chain = RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="question",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
)

In [6]:
# Retriever 문서 점검, 리트리버 조정 및 문서검색 확인

# faiss_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})
# bm25_retriever = BM25Retriever.from_documents(combined_documents, preprocess_func=kiwi_tokenize, k=10)

# ensemble_retriever = EnsembleRetriever(
#     retrievers=[bm25_retriever, faiss_retriever],
#     weights=[0.4, 0.6], search_type="mmr",
# )

query = "Prompt Temlate에 대해 설명해줘!"
documents = ensemble_retriever.invoke(query)
print("[ensemble_retriver]")
for doc in documents : print(doc.metadata['source'])
# for doc in documents : print(doc), print("\n")

# print("\n[faiss_retriver]")
# documents = faiss_retriever.invoke(query)
# for doc in documents : print(doc.metadata['source'])

# print("\n[chroma_retriver]")
# documents = chroma_retriever.invoke(query)
# for doc in documents : print(doc.metadata['source'])

[ensemble_retriver]
https://wikidocs.net/234017
https://wikidocs.net/234475
https://wikidocs.net/234018
https://wikidocs.net/234021
https://wikidocs.net/235703
https://wikidocs.net/234018
https://wikidocs.net/233325
https://wikidocs.net/234018
https://wikidocs.net/234475
https://wikidocs.net/235886
https://wikidocs.net/233351
https://wikidocs.net/233350
https://wikidocs.net/234020
https://wikidocs.net/233790
https://wikidocs.net/233350
https://wikidocs.net/235704
https://wikidocs.net/234021
https://wikidocs.net/233791
https://wikidocs.net/233798
https://wikidocs.net/233804


In [7]:
response = rag_chain.invoke({"question": query}, config={"configurable": {"session_id": "foo"}})

foo
Prompt Template은 자연어 처리(NLP) 모델, 특히 언어 모델을 사용할 때 입력으로 제공되는 텍스트의 형식을 정의하는 템플릿입니다. Prompt Template은 모델이 특정 문맥에서 작동하도록 설정하고, 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성할 수 있도록 돕습니다. Prompt Template은 주로 다음과 같은 요소로 구성됩니다:

1. **지시사항(Instruction)**: 모델에게 주어진 작업을 설명하는 부분입니다. 예를 들어, "질문에 답하세요" 또는 "다음 문장을 번역하세요"와 같은 지시사항이 포함될 수 있습니다.

2. **질문(Question)**: 사용자가 입력한 질문이나 명령입니다. 이 부분은 모델이 처리해야 할 실제 입력입니다.

3. **문맥(Context)**: 모델이 질문에 답하기 위해 사용할 수 있는 추가 정보입니다. 이는 검색된 문서나 데이터베이스에서 가져온 정보일 수 있습니다.

### 예제
다음은 Prompt Template의 예제입니다:

```python
template = """
당신은 질문-답변(Question-Answer) Task를 수행하는 AI 어시스턴트입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요.
만약, 문맥(context)으로부터 답을 찾을 수 없다면 '모른다'고 말하세요.
한국어로 대답하세요.

# Question: {question}
# Context: {context}
"""
```

이 템플릿은 모델에게 질문과 문맥을 제공하고, 이를 바탕으로 답변을 생성하도록 지시합니다.

### 사용 예제
Prompt Template을 사용하여 실제로 모델을 호출하는 예제는 다음과 같습니다:

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 템플릿을 정의합니다.
template = """
당신은 질

In [8]:
response = rag_chain.invoke({"question": "조금전에 이야기한 1번에 대해 설명해줘!"}, config={"configurable": {"session_id": "foo"}})

foo
1번 항목인 **지시사항(Instruction)**에 대해 자세히 설명드리겠습니다.

### 지시사항(Instruction)

지시사항은 Prompt Template의 핵심 요소 중 하나로, 모델에게 주어진 작업을 명확하게 설명하는 부분입니다. 지시사항은 모델이 특정 작업을 수행할 때 필요한 지침을 제공하며, 모델이 올바른 방식으로 응답을 생성할 수 있도록 돕습니다. 지시사항은 다음과 같은 역할을 합니다:

1. **작업의 명확화**: 모델이 수행해야 할 작업을 명확하게 정의합니다. 예를 들어, 질문에 답변하는 것인지, 텍스트를 번역하는 것인지, 요약하는 것인지 등을 명확히 합니다.
2. **문맥 설정**: 모델이 어떤 문맥에서 작동해야 하는지 설정합니다. 이를 통해 모델은 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성할 수 있습니다.
3. **응답 형식 지정**: 모델이 생성해야 할 응답의 형식을 지정합니다. 예를 들어, 응답이 간결해야 하는지, 특정 형식을 따라야 하는지 등을 지시할 수 있습니다.

### 예제

다음은 지시사항을 포함한 Prompt Template의 예제입니다:

```python
template = """
당신은 질문-답변(Question-Answer) Task를 수행하는 AI 어시스턴트입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요.
만약, 문맥(context)으로부터 답을 찾을 수 없다면 '모른다'고 말하세요.
한국어로 대답하세요.

# Question: {question}
# Context: {context}
"""
```

이 템플릿에서 지시사항은 다음과 같습니다:
- "당신은 질문-답변(Question-Answer) Task를 수행하는 AI 어시스턴트입니다."
- "검색된 문맥(context)를 사용하여 질문(question)에 답하세요."
- "만약, 문맥(context)으로부터 답을 찾을 수 없다면 '모른다'고 말하세요."
- "한국어로 대답하세요."

이 지시사항들은